In [5]:
import copy

import numpy as np

def parse_board(board):
    return np.array([[int(c) if c.isdigit() else 0 for c in row.strip() if c != " "] for row in board.strip().split('\n')])

def print_board(board):
    for i, row in enumerate(board):
        if i % 3 == 0 and i != 0:
            print("- - - - - - - - - - - -")
        for j, val in enumerate(row):
            if j % 3 == 0 and j != 0:
                print("|", end=" ")
            if j == 8:
                print(val)
            else:
                print(str(val) + " ", end="")
    print()

def board_is_solved(board):

    for num in range(1, 10):
        # Check rows
        for i in range(9):
            count = 0
            for j in range(9): #cycle through columns of a fixed row
                if board[i][j] == num:
                    count += 1
                if count > 1: #if a number appear more than once in a row
                    return False
            if count < 1: #if a number doesn't appear in a row
                return False
                    
        # check columns
        for j in range(9):
            count = 0
            for i in range(9): #cycle through rows of a fixed column
                if board[i][j] == num:
                    count += 1
                if count > 1: #if a number appear more than once in a row
                    return False
            if count < 1: #if a number doesn't appear in a row
                return False

    # Check boxes
        for box_x in range(1, 3):
            for box_y in range(1, 3):
                count = 0
                for i in range(box_y * 3, box_y * 3 + 3):
                    for j in range(box_x * 3, box_x * 3 + 3):
                        if board[i][j] == num:
                            count += 1
                        if(count > 1): #if a number appears more than one time in a box 3x3
                            return False
                if count < 1: #if a number doesn't appear in a box
                    return False
    return True


def board_is_full(board):
    
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:
                return False
    return True

In [6]:
import random
from multiprocessing.pool import ThreadPool as Pool

# Genetic Algorithm parameters
POPULATION_SIZE = 1000
MUTATION_RATE = 0.2
MAX_GENERATIONS = 100000
PERCENTAGE_ELITISM = 0.2

# generate a new board with random numbers (where numbers were not already set)
def generate_board_state(initial_board):
    
    initial = copy.deepcopy(initial_board)
    for i in range(9):
        gene = list()
        # create a list for every row with the numbers from 1 to 9, removing the numbers already present in the row
        for j in range(1,10):
            if not j in initial_board[i]:
                gene.append(j)
        random.shuffle(gene)
        
        for j in range(9):
            if initial_board[i][j] != 0:
                initial[i][j] = initial_board[i][j]
            else:
                initial[i][j] = gene.pop()
    
    return initial

def coords_already_full(board):
    
    list = []
    
    for i in range(9):
        for j in range(9):
            if board[i][j] != 0:
                list.append((i,j))
    
    return list

def fitness(board):
    fitness_score = 0
    
    for j in range(9): #for each column
        seen = {}
        for i in range(9): # Check each cell in the column
            if board[i][j] in seen:
                seen[board[i][j]] += 1
            else:
                seen[board[i][j]] = 1
        for key in seen: # Subtract fitness for repeated numbers
            fitness_score -= (seen[key] - 1)
    
    for m in range(3): # For each 3x3 square
        for n in range(3):
            seen = {}
            for i in range(3 * n, 3 * (n + 1)):  # Check cells in 3x3 square
                for j in range(3 * m, 3 * (m + 1)):
                    if board[i][j] in seen:
                        seen[board[i][j]] += 1
                    else:
                        seen[board[i][j]] = 1
            for key in seen: # Subtract fitness for repeated numbers
                fitness_score -= (seen[key] - 1)
    
    return fitness_score

# Select parents for crossover using roulette wheel selection
def roulette_wheel_selection(population, already_parent = None):
    # Calculate total fitness sum of the population
    total_fitness = -sum(individual[1] for individual in population)
    
    #print(total_fitness)
    if already_parent is not None:
        total_fitness += already_parent[1]

    # Generate a random number between 0 and the total fitness sum
    spin = random.uniform(0, total_fitness)

    # Iterate through the population and sum fitness values until the spin value is reached
    fitness_sum = 0
    for individual in population:
        # doesn't take 2 equal parents
        if not np.array_equal(individual, already_parent):
            fitness_sum += -individual[1]
            if fitness_sum >= spin:
                return individual
            
def crossover(parent1, parent2):
    
    selected = random.randint(1,2)
    crossover_point = random.randint(1, 8)
    
    child = copy.deepcopy(parent2)
    
    for i in range(crossover_point):
        child[i] = parent1[i]
            
    return child
        
def mutate(board, already_filled):
    
    row = random.randint(0, 8)
    
    while True:
        pos1 = random.randint(0, 8)
        pos2 = random.randint(0, 8)
        if not (row,pos1) in already_filled and not (row,pos2) in already_filled and pos1 != pos2:
            break
    
    new_board = copy.deepcopy(board)
    temp = new_board[row][pos1]
    new_board[row][pos1] = new_board[row][pos2]
    new_board[row][pos2] = temp
    
    return new_board

In [7]:
# Example Board
easy_1 = """
37. 5....6
... 36. .12
... .91 75.
... 154 .7.
..3 .7. 6..
.5. 638 ...
.64 98. ...
59. .26 ...
2.. ..5 .64
"""

sudoku = """
8........
..36.....
.7..9.2..
.5...7...
....45.7.
...1...3.
..1....68
..85...1.
.9....4..
"""

easy_2 = """
.7. ..4 13.
... 2.7 ..6
..5 .13 .2.
..1 ..2 ...
..2 19. .57
..3 .45 8.2
.1. 378 26.
367 ... 58.
8.9 ..1 .7.
"""

### multiprocessing

In [ ]:
board = parse_board(easy_1)

# Generate the initial population
population = [(generate_board_state(board), 0) for _ in range(POPULATION_SIZE)]

# save in a var the coordinates to the cells already filled
already_filled = coords_already_full(board)

# set the best board to the initial board
best_board_state = board
NUM_CORE = 8

# Main Genetic Algorithm loop
for generation in range(MAX_GENERATIONS):
    # Calculate fitness for each board state
    
    pool = Pool(NUM_CORE)
    fitnesses = pool.map(fitness, [board_state for board_state, _ in population])
    pool.close()
    pool.join()
    population = list(zip([board_state for board_state, _ in population], fitnesses))
    
    random.shuffle(population)
    
    # Check if solution is found
    best_board_state = max(population, key=lambda x: x[1])[0]
    if fitness(best_board_state) == 0:
        print("Solution found in generation", generation)
        break
        
    ### PRINT STATE
    print("Generazione ", generation)
    print("Fitness: ", max(population, key=lambda x: x[1])[1])
    print_board(best_board_state)
    print()
        
    # Create the next generation
    new_population = []

    population_copy = population.copy()
    # Elitism: Keep the best board state from the previous generation
    for n in range(int(PERCENTAGE_ELITISM * POPULATION_SIZE)):
        
        max_individual = max(population_copy, key=lambda x: x[1])
        if not any(np.array_equal(max_individual[0], existing[0]) for existing in new_population):
            new_population.append(max_individual)
        population_copy = [x for x in population_copy if not np.array_equal(x[0], max_individual[0])]
    
    # Perform selection, crossover, and mutation
    while len(new_population) < POPULATION_SIZE:
        parent1 = roulette_wheel_selection(population)
        # Select the other parent without the change that can take the parent1
        parent2 = roulette_wheel_selection(population, parent1)
        
        child = crossover(parent1[0], parent2[0])
        if random.random() < MUTATION_RATE:
            child = mutate(child, already_filled)
        
        # Optimization: insert only different childs
        if not any(np.array_equal(child, existing[0]) for existing in new_population):
            new_population.append((child, 0))

    # Update the population
    population = new_population
    
# Print the best solution
print("Best solution:", best_board_state)
if(board_is_solved(best_board_state)):
    print("Board risolta")

Generazione  0
Fitness:  -29
3 7 8 | 5 2 1 | 4 9 6
5 4 9 | 3 6 8 | 7 1 2
3 2 6 | 4 9 1 | 7 5 8
- - - - - - - - - - - -
2 8 9 | 1 5 4 | 3 7 6
8 1 3 | 2 7 9 | 6 4 5
9 5 7 | 6 3 8 | 4 2 1
- - - - - - - - - - - -
7 6 4 | 9 8 1 | 2 5 3
5 9 4 | 7 2 6 | 1 3 8
2 1 3 | 8 9 5 | 7 6 4


Generazione  1
Fitness:  -29
3 7 8 | 5 2 1 | 4 9 6
5 4 9 | 3 6 8 | 7 1 2
3 2 6 | 4 9 1 | 7 5 8
- - - - - - - - - - - -
2 8 9 | 1 5 4 | 3 7 6
8 1 3 | 2 7 9 | 6 4 5
9 5 7 | 6 3 8 | 4 2 1
- - - - - - - - - - - -
7 6 4 | 9 8 1 | 2 5 3
5 9 4 | 7 2 6 | 1 3 8
2 1 3 | 8 9 5 | 7 6 4


Generazione  2
Fitness:  -28
3 7 8 | 5 2 1 | 4 9 6
5 4 9 | 3 6 8 | 7 1 2
3 2 6 | 4 9 1 | 7 5 8
- - - - - - - - - - - -
2 8 9 | 1 5 4 | 3 7 6
8 1 3 | 2 7 9 | 6 4 5
9 5 7 | 6 3 8 | 4 2 1
- - - - - - - - - - - -
7 6 4 | 9 8 2 | 5 1 3
5 9 8 | 7 2 6 | 1 3 4
2 3 9 | 8 1 5 | 7 6 4


Generazione  3
Fitness:  -28
3 7 8 | 5 2 1 | 4 9 6
5 4 9 | 3 6 8 | 7 1 2
3 2 6 | 4 9 1 | 7 5 8
- - - - - - - - - - - -
2 8 9 | 1 5 4 | 3 7 6
8 1 3 | 2 7 9 | 6 4 5
9 5 7 

In [5]:
board = parse_board(easy_2)

# Generate the initial population
population = [(generate_board_state(board), 0) for _ in range(POPULATION_SIZE)]

# save in a var the coordinates to the cells already filled
already_filled = coords_already_full(board)

# set the best board to the initial board
best_board_state = board

# Main Genetic Algorithm loop
for generation in range(MAX_GENERATIONS):
    # Calculate fitness for each board state
    population = [(board_state, fitness(board_state)) for board_state, _ in population]
    
    random.shuffle(population)
    
    # Check if solution is found
    best_board_state = max(population, key=lambda x: x[1])[0]
    if fitness(best_board_state) == 0:
        print("Solution found in generation", generation)
        break
        
    ### PRINT STATE
    print("Generazione ", generation)
    print("Fitness: ", max(population, key=lambda x: x[1])[1])
    print_board(best_board_state)
    print()
        
    # Create the next generation
    new_population = []

    population_copy = population.copy()
    # Elitism: Keep the best board state from the previous generation
    for n in range(int(PERCENTAGE_ELITISM * POPULATION_SIZE)):
        
        max_individual = max(population_copy, key=lambda x: x[1])
        if not any(np.array_equal(max_individual[0], existing[0]) for existing in new_population):
            new_population.append(max_individual)
        population_copy = [x for x in population_copy if not np.array_equal(x[0], max_individual[0])]
    
    # Perform selection, crossover, and mutation
    while len(new_population) < POPULATION_SIZE:
        parent1 = roulette_wheel_selection(population)
        # Select the other parent without the change that can take the parent1
        parent2 = roulette_wheel_selection(population, parent1)
        
        child = crossover(parent1[0], parent2[0])
        if random.random() < MUTATION_RATE:
            child = mutate(child, already_filled)
        
        # Optimization: insert only different childs
        if not any(np.array_equal(child, existing[0]) for existing in new_population):
            new_population.append((child, 0))

    # Update the population
    population = new_population
    
# Print the best solution
print("Best solution:", best_board_state)
if(board_is_solved(best_board_state)):
    print("Board risolta")

Generazione  0
Fitness:  -54
6 7 9 | 7 1 4 | 1 3 8
1 9 5 | 2 6 7 | 4 8 6
7 4 5 | 7 1 3 | 3 2 3
- - - - - - - - - - - -
7 2 1 | 8 3 2 | 6 9 4
8 3 2 | 1 9 6 | 4 5 7
4 8 3 | 7 4 5 | 8 5 2
- - - - - - - - - - - -
5 1 9 | 3 7 8 | 2 6 1
3 6 7 | 4 3 9 | 5 8 8
8 9 9 | 7 9 1 | 3 7 4




KeyboardInterrupt: 

In [33]:
list = [(0,0), (0,1), (1,0), (1,1), (1,1)]
print(list)
list.remove(max(list, key=lambda x: x[1]))
print(list)
list.remove(max(list, key=lambda x: x[1]))
print(list)

for i in range(8):
    print(i)

[(0, 0), (0, 1), (1, 0), (1, 1), (1, 1)]
[(0, 0), (1, 0), (1, 1), (1, 1)]
[(0, 0), (1, 0), (1, 1)]
0
1
2
3
4
5
6
7


In [40]:
list = [1,2,3,4,5,6,7,8,9]
element = 1
if not any((element == x) for x in list):
    print("non presente")